### Initial acessment

In [1]:
# pip install selenium webdriver-manager

import pandas as pd
import json
import numpy as np
import os

from utilities.funcoes import extrair_ids
from utilities.funcoes_scrape import extract_archives_under, download_pecas_flat, keep_only_key_docs, download_contract_pdfs

c:\Users\carol\Documents\bolsa\RISKGUARD\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
contratos = pd.read_csv("../data/impic_data/contratos.csv", sep=';')
contratos_niclas = pd.read_parquet(r"..\data\contracts_2009_2024.parquet")

C:\Users\carol\AppData\Local\Temp\ipykernel_20408\3211104189.py:1: DtypeWarning: Columns (18,19,23,24,25,26,27,28,29,30,31,32,33,34,35,38,39,40,41,43,44,52,54) have mixed types. Specify dtype option on import or set low_memory=False.
  contratos = pd.read_csv("../data/impic_data/contratos.csv", sep=';')


In [3]:
# colocar as datas relevantes para a análise em datetime
contratos_niclas['close_date']=pd.to_datetime(contratos_niclas['close_date'], errors='coerce')
contratos_niclas['signing_date'] = pd.to_datetime(contratos_niclas['signing_date'], errors='coerce')
contratos['Data Celebração']=pd.to_datetime(contratos['Data Celebração'], errors='coerce')

# extrair o id dos documentos
contratos_niclas['ids_extraidos_docs'] = contratos_niclas['documents'].apply(extrair_ids)

In [4]:
# ver os contratos que já têm o link para as Peças do Procedimento válido
data = []
with open("../data/contracts_links_OT.jsonl", 'r', encoding='utf-8') as file:
    for line in file:
        data.append(json.loads(line))
contracts_scraped = pd.DataFrame(data)

In [5]:
len(contracts_scraped)

3600

In [6]:
### Para cada um destes casos há um tratamento diferente

# contratos que têm link direto para download
contracts_download = contracts_scraped[contracts_scraped['pecas_link'].str.contains('donwload', case=False, na=False, regex=True)]
# contratos que têm link para o vortal
contracts_vortal = contracts_scraped[contracts_scraped['pecas_link'].str.contains('vortal', case=False, na=False, regex=True)]
# contratos que foram scraped até à data
contratos_niclas_scraped = contratos_niclas[(contratos_niclas['contract_id'].isin(contracts_scraped['contract_id'].to_list()))]

In [8]:
len(contracts_download)

1378

In [7]:
len(contracts_vortal)

1606

In [9]:
# objetivo é ver as pastas que já foram criadas e exluir estes ids da extração por download do pdf e do anuncio
# vou deixar a extração com o vortal para o fim, por isso todos os que tiverem donwload têm de ser passados ainda pelo vortal
# todos os que estiverem nesta list não sáo passados pelo download nem pela extraçao do anuncio e do pdf do contrato, mas têm de ser passados pelo vortal se for esse o caso
ids_extraidos = []
for item in os.listdir('../docs2'):
    ids_extraidos.append(item)

In [10]:
# todos os que têm pasta eu já tentei fazer o download se têm download e já fiz também anuncio e contrato
# vou acrescentar aqui depois o do vortal, quando tiver tudo certo
# ou então faço a extração com o caso do vortal noutro notebook
not_extracted = contracts_scraped[~(contracts_scraped['contract_id'].apply(lambda x: str(x)).isin(ids_extraidos))]
not_extracted_download = not_extracted[not_extracted['pecas_link'].str.contains('donwload', case=False, na=False, regex=True)]

In [12]:
len(not_extracted_download)

503

### Caso vortal

Ainda tenho de melhorar este caso - fica para mais tarde, primeiro desenvolver a pipeline que já tenho

In [16]:
# pip install selenium webdriver-manager requests

import re, time, unicodedata
from pathlib import Path
import urllib.parse as up

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Selectors (adjust if your tenant’s DOM differs) ---
VORTAL_DOC_NAME = "span.VortalSpan[id*='spnDocumentName']"
VORTAL_DL_LINK  = "a[id^='lnkDownloadLink']"

# ---------- Helpers ----------
def _norm(s: str) -> str:
    if not s:
        return ""
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = s.lower()
    s = re.sub(r"[\s_]+", " ", s).strip()  # turn "_" into spaces and collapse runs
    return s

def _want(name: str, type: str) -> bool:
    t = _norm(name)
    # keep: "caderno ... encargos" OR "programa" OR "ce"
    if type=='caderno':
        return (('caderno' in t and "encargos" in t) or ("ce " in t) and ('programa' not in t) and ('anexo' not in t))
    elif type=='programa':
        return('programa' in t)
    else:
        raise ValueError(f"type inválido: {type!r}. Esperado: 'caderno' ou 'programa'.")
    

def _allow_downloads(drv, out_dir: Path):
    try:
        drv.execute_cdp_cmd("Page.setDownloadBehavior", {"behavior": "allow", "downloadPath": str(out_dir)})
    except Exception:
        pass

def _accept_cookies_if_present(drv):
    for how, sel in [
        (By.ID, "onetrust-accept-btn-handler"),
        (By.XPATH, "//button[contains(., 'Aceitar')]"),
        (By.XPATH, "//button[contains(., 'Accept')]"),
        (By.XPATH, "//button[contains(., 'I Agree')]"),
        (By.XPATH, "//div[contains(@class,'cookie')]//button"),
    ]:
        try:
            btn = WebDriverWait(drv, 3).until(EC.element_to_be_clickable((how, sel)))
            drv.execute_script("arguments[0].click();", btn)
            time.sleep(0.2)
            break
        except Exception:
            continue

def _open_documents_tab(drv):
    for xp in (
        "//a[normalize-space()='Documentos']", "//button[normalize-space()='Documentos']",
        "//a[normalize-space()='Documents']",  "//button[normalize-space()='Documents']",
        "//div[contains(@class,'tab')]//a[contains(.,'Documentos')]",
        "//div[contains(@class,'tab')]//a[contains(.,'Documents')]",
    ):
        try:
            el = drv.find_element(By.XPATH, xp)
            drv.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
            time.sleep(0.1)
            drv.execute_script("arguments[0].click();", el)
            time.sleep(0.2)
            return True
        except Exception:
            pass
    return False

def _scroll_page(drv, steps=6, pause=0.25):
    last_h = 0
    for _ in range(steps):
        drv.execute_script("window.scrollBy(0, document.body.scrollHeight);")
        time.sleep(pause)
        h = drv.execute_script("return document.body.scrollHeight")
        if h == last_h: break
        last_h = h

def _parse_onclick_concat(onclick: str):
    """Collapse '...' + '...' into one string, then extract query params for DownloadFile."""
    if not onclick: return None, None
    s = re.sub(r"'\s*\+\s*'", "", onclick)
    s = s.replace("'+ '", "").replace("' +", "")
    m = re.search(r"DownloadFile\?([^'\" ]+)", s)
    if not m: return None, None
    qd = dict(up.parse_qsl(m.group(1)))
    return qd.get("documentFileId"), qd.get("mkey")

def _downloads_done(dirpath: Path) -> bool:
    return not any(p.suffix == ".crdownload" for p in dirpath.glob("*"))

# ---------- Main ----------
def download_vortal_selected_via_browser(row,type: str, out_root: str, headless: bool = False, timeout: int = 60, tries: int = 3):
    out_dir = Path(out_root) / str(row['id'])
    out_dir.mkdir(parents=True, exist_ok=True)
    out_dir = Path(out_dir).resolve()
    out_dir.mkdir(parents=True, exist_ok=True)

    prefs = {
        "download.default_directory": str(out_dir),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "plugins.always_open_pdf_externally": True,  # force saving PDF (no in-tab viewer)
    }
    opts = webdriver.ChromeOptions()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1280,900")
    opts.add_argument("--lang=pt-PT")
    opts.add_argument("--disable-popup-blocking")
    opts.add_experimental_option("prefs", prefs)

    drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    try:
        _allow_downloads(drv, out_dir)

        # load + prep page (with retries for lazy load / banners / tabs)
        for attempt in range(1, tries+1):
            drv.get(row['link'])
            WebDriverWait(drv, timeout).until(lambda d: d.execute_script("return document.readyState") == "complete")
            _accept_cookies_if_present(drv)
            _open_documents_tab(drv)
            _scroll_page(drv)

            try:
                WebDriverWait(drv, timeout).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, VORTAL_DOC_NAME)))
                if drv.find_elements(By.CSS_SELECTOR, VORTAL_DOC_NAME):
                    break
            except Exception:
                if attempt == tries:
                    # dump debug
                    (out_dir/"vortal_debug.png").write_bytes(drv.get_screenshot_as_png())
                    (out_dir/"vortal_debug.html").write_text(drv.page_source, encoding="utf-8")
                    return {"ok": False, "reason": "doc_names_not_found",
                            "debug": [str(out_dir/"vortal_debug.png"), str(out_dir/"vortal_debug.html")]}
                time.sleep(1.5 * attempt)

        # Map index suffix -> visible name
        idx_to_name = {}
        for s in drv.find_elements(By.CSS_SELECTOR, VORTAL_DOC_NAME):
            sid = s.get_attribute("id") or ""
            m = re.search(r"_(\d+)$", sid)
            idx = m.group(1) if m else sid
            idx_to_name[idx] = (s.text or "").strip()

        # Collect anchors and select only the wanted ones
        targets = []
        for a in drv.find_elements(By.CSS_SELECTOR, VORTAL_DL_LINK):
            aid = a.get_attribute("id") or ""
            m = re.search(r"_(\d+)$", aid)
            idx = m.group(1) if m else aid
            name = idx_to_name.get(idx, "").strip()
            if name and _want(name, type=type):
                targets.append((name, a))

        if not targets:
            return {"ok": True, "matched": 0, "files": [], "note": "no matching names"}

        # Build direct download URLs and navigate to them (keeps browser context → no 403)
        base = f"{up.urlparse(row['link']).scheme}://{up.urlparse(row['link']).netloc}"
        before = {p.name for p in out_dir.glob("*") if p.is_file()}

        for name, a in targets:
            onclick = a.get_attribute("onclick") or ""
            doc_id, mkey = _parse_onclick_concat(onclick)
            if doc_id and mkey:
                dl_url = f"{base}/PRODPublic/Tendering/OpportunityDetail/DownloadFile?documentFileId={doc_id}&mkey={mkey}"
                drv.get(dl_url)  # triggers true download
            else:
                # fallback: click
                drv.execute_script("arguments[0].scrollIntoView({block:'center'});", a)
                drv.execute_script("arguments[0].click();", a)
            time.sleep(0.6)

        # wait for downloads to finish
        t0 = time.time()
        max_wait = max(30, 15 * len(targets))
        while time.time() - t0 < max_wait:
            time.sleep(0.5)
            if _downloads_done(out_dir):
                break

        after = {p.name for p in out_dir.glob("*") if p.is_file()}
        new_files = sorted(list(after - before))
        return {"ok": True, "matched": len(targets), "files": [str(out_dir / f) for f in new_files]}

    finally:
        drv.quit()

In [50]:
# --- keep your imports ---
import json, html
import re, time, unicodedata
from pathlib import Path
import urllib.parse as up

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ... keep your existing constants and helpers ...
# i am going to match only a document each time

def _norm(s: str) -> str:
    if not s:
        return ""
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = s.lower()
    s = re.sub(r"[\s_]+", " ", s).strip()
    return s

def _want(name: str, type: str) -> bool:
    """
    Heurística simples para 'caderno' ou 'programa'.
    - 'caderno': contém 'caderno' e 'encargos' ou é 'ce' isolado; exclui 'programa' e 'anexo'
    - 'programa': contém 'programa'
    """
    t = _norm(name)
    if type == 'caderno':
        is_ce = re.search(r"\bce\b", t) is not None
        ok = ('caderno' in t or 'encargos' in t or is_ce)
        bad = ('programa' in t) or ('anexo' in t)
        return ok and not bad
    elif type == 'programa':
        return ('programa' in t)
    else:
        raise ValueError(f"type inválido: {type!r}. Esperado: 'caderno' ou 'programa'.")

def _parse_onclick_concat(onclick: str):
    """
    Collapse string concatenations in onclick and extract query params for DownloadFile.
    Returns (documentFileId, mkey) or (None, None).
    """
    if not onclick:
        return None, None
    s = html.unescape(onclick)  # decode &amp;
    s = re.sub(r"'\s*\+\s*'", "", s)
    s = s.replace("'+ '", "").replace("' +", "")
    m = re.search(r"DownloadFile\?([^'\" ]+)", s)
    if not m:
        return None, None
    qd = dict(up.parse_qsl(m.group(1)))
    return qd.get("documentFileId"), qd.get("mkey")

def _downloads_done(dirpath: Path) -> bool:
    return not any(p.suffix == ".crdownload" for p in dirpath.glob("*"))

def _load_ledger(out_dir: Path) -> set:
    """Keep a tiny JSON ledger of already-downloaded documentFileId values."""
    led = out_dir / "downloaded.json"
    if led.exists():
        try:
            return set(json.loads(led.read_text(encoding="utf-8")))
        except Exception:
            return set()
    return set()

def _save_ledger(out_dir: Path, ids: set):
    (out_dir / "downloaded.json").write_text(json.dumps(sorted(ids)), encoding="utf-8")

def download_vortal_selected_via_browser(row, type: str, out_root: str,
                                         headless: bool = False, timeout: int = 60,
                                         tries: int = 3, max_files: int = 1):
    """
    Downloads up to `max_files` matching documents (default: 1) for this contract row.
    Prevents re-downloads by keeping a ledger of documentFileIds in downloaded.json.
    """


    VORTAL_DOC_NAME = "span.VortalSpan[id*='spnDocumentName']"
    VORTAL_DL_LINK  = "a[id^='lnkDownloadLink']"

    out_dir = Path(out_root) / str(row['id'])
    out_dir.mkdir(parents=True, exist_ok=True)
    out_dir = Path(out_dir).resolve()

    # Load ledger (already downloaded documentFileIds)
    ledger = _load_ledger(out_dir)

    prefs = {
        "download.default_directory": str(out_dir),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "plugins.always_open_pdf_externally": True,
    }
    opts = webdriver.ChromeOptions()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1280,900")
    opts.add_argument("--lang=pt-PT")
    opts.add_argument("--disable-popup-blocking")
    opts.add_experimental_option("prefs", prefs)

    drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    try:
        # Allow downloads via CDP (best-effort)
        try:
            drv.execute_cdp_cmd("Page.setDownloadBehavior", {"behavior": "allow", "downloadPath": str(out_dir)})
        except Exception:
            pass

        # load + prep page (with retries)
        for attempt in range(1, tries+1):
            drv.get(row['link'])
            WebDriverWait(drv, timeout).until(lambda d: d.execute_script("return document.readyState") == "complete")

            # cookies
            for how, sel in [
                (By.ID, "onetrust-accept-btn-handler"),
                (By.XPATH, "//button[contains(., 'Aceitar')]"),
                (By.XPATH, "//button[contains(., 'Accept')]"),
                (By.XPATH, "//button[contains(., 'I Agree')]"),
                (By.XPATH, "//div[contains(@class,'cookie')]//button"),
            ]:
                try:
                    btn = WebDriverWait(drv, 3).until(EC.element_to_be_clickable((how, sel)))
                    drv.execute_script("arguments[0].click();", btn)
                    time.sleep(0.2)
                    break
                except Exception:
                    continue

            # open "Documentos" tab
            opened = False
            for xp in (
                "//a[normalize-space()='Documentos']", "//button[normalize-space()='Documentos']",
                "//a[normalize-space()='Documents']",  "//button[normalize-space()='Documents']",
                "//div[contains(@class,'tab')]//a[contains(.,'Documentos')]",
                "//div[contains(@class,'tab')]//a[contains(.,'Documents')]",
            ):
                try:
                    el = drv.find_element(By.XPATH, xp)
                    drv.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
                    time.sleep(0.1)
                    drv.execute_script("arguments[0].click();", el)
                    time.sleep(0.2)
                    opened = True
                    break
                except Exception:
                    pass

            # scroll to load
            last_h = 0
            for _ in range(6):
                drv.execute_script("window.scrollBy(0, document.body.scrollHeight);")
                time.sleep(0.25)
                h = drv.execute_script("return document.body.scrollHeight")
                if h == last_h: break
                last_h = h

            try:
                WebDriverWait(drv, timeout).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, VORTAL_DOC_NAME)))
                if drv.find_elements(By.CSS_SELECTOR, VORTAL_DOC_NAME):
                    break
            except Exception:
                if attempt == tries:
                    (out_dir/"vortal_debug.png").write_bytes(drv.get_screenshot_as_png())
                    (out_dir/"vortal_debug.html").write_text(drv.page_source, encoding="utf-8")
                    return {"ok": False, "reason": "doc_names_not_found",
                            "debug": [str(out_dir/'vortal_debug.png'), str(out_dir/'vortal_debug.html')]}
                time.sleep(1.5 * attempt)

        # Map index suffix -> visible name
        idx_to_name = {}
        for s in drv.find_elements(By.CSS_SELECTOR, VORTAL_DOC_NAME):
            sid = s.get_attribute("id") or ""
            m = re.search(r"_(\d+)$", sid)
            idx = m.group(1) if m else sid
            idx_to_name[idx] = (s.text or "").strip()

        # Collect anchors and pair with their doc ids via onclick
        candidates = []
        for a in drv.find_elements(By.CSS_SELECTOR, VORTAL_DL_LINK):
            aid = a.get_attribute("id") or ""
            m = re.search(r"_(\d+)$", aid)
            idx = m.group(1) if m else aid
            name = idx_to_name.get(idx, "").strip()
            if not name:
                continue
            if not _want(name, type=type):
                continue
            onclick = a.get_attribute("onclick") or ""
            doc_id, mkey = _parse_onclick_concat(onclick)
            candidates.append({"name": name, "a": a, "doc_id": doc_id, "mkey": mkey})

        # Remove ones already downloaded (using the ledger)
        targets = [c for c in candidates if c["doc_id"] and c["doc_id"] not in ledger]

        if not targets:
            return {"ok": True, "matched": 0, "files": [], "note": "no new matching names (all already downloaded or none match)"}

        # First-match only (or up to max_files)
        targets = targets[:max_files]

        # Prepare before/after listing
        before = {p.name for p in out_dir.glob("*") if p.is_file()}

        # Build direct URL when possible, else click
        base = f"{up.urlparse(row['link']).scheme}://{up.urlparse(row['link']).netloc}"
        for c in targets:
            if c["doc_id"] and c["mkey"]:
                dl_url = f"{base}/PRODPublic/Tendering/OpportunityDetail/DownloadFile?documentFileId={c['doc_id']}&mkey={c['mkey']}"
                drv.get(dl_url)
            else:
                drv.execute_script("arguments[0].scrollIntoView({block:'center'});", c["a"])
                drv.execute_script("arguments[0].click();", c["a"])
            time.sleep(0.7)

        # wait for downloads to finish
        t0 = time.time()
        max_wait = max(30, 15 * len(targets))
        while time.time() - t0 < max_wait:
            time.sleep(0.5)
            if _downloads_done(out_dir):
                break

        after = {p.name for p in out_dir.glob("*") if p.is_file()}
        new_files = sorted(list(after - before))

        # Update ledger with the doc_ids we attempted (only if new files appeared)
        for c in targets:
            if c["doc_id"]:
                ledger.add(c["doc_id"])
        _save_ledger(out_dir, ledger)

        return {"ok": True, "matched": len(targets), "files": [str(out_dir / f) for f in new_files]}

    finally:
        drv.quit()

In [51]:
# tentar extrair o caderno de encargos
# problema - muitas vezes vem como CE, acho que já resolvi o problema, não sei se está à prova de bala
contracts_vortal.apply(download_vortal_selected_via_browser, axis=1, type = 'caderno', headless=False, timeout=60, out_root='../docs')

1                {'ok': True, 'matched': 1, 'files': []}
2      {'ok': True, 'matched': 1, 'files': ['C:\Users...
3      {'ok': True, 'matched': 1, 'files': ['C:\Users...
4      {'ok': True, 'matched': 1, 'files': ['C:\Users...
6      {'ok': True, 'matched': 1, 'files': ['C:\Users...
                             ...                        
792    {'ok': True, 'matched': 1, 'files': ['C:\Users...
794    {'ok': True, 'matched': 1, 'files': ['C:\Users...
795    {'ok': True, 'matched': 1, 'files': ['C:\Users...
797    {'ok': True, 'matched': 1, 'files': ['C:\Users...
799    {'ok': True, 'matched': 1, 'files': ['C:\Users...
Length: 387, dtype: object

In [ ]:
contracts_vortal.apply(download_vortal_selected_via_browser, axis=1, type = 'programa', headless=False, timeout=60, out_root='../docs')

### Caso download direto

#### Fazer o download da pasta

In [14]:
saved = contracts_download.apply(download_pecas_flat, axis=1, out_root="../docs2")

In [ ]:
# download peças do procedimento folders
saved2 = not_extracted_download.apply(download_pecas_flat, axis=1, out_root="../docs2")

#### Unzip Folders that were ziped

In [ ]:
# unzip downloaded folders inside the main folder

summary = extract_archives_under(Path("../docs2"), max_zip_depth=4, recursive=False)
print("Processed:", len(summary["processed"]), "archives")
print("Errors:", len(summary["errors"]))

Processed: 107 archives
Errors: 0


#### Filtrar só pelos documentos que me interessam - CE e programa

In [24]:
resultados = []

folder_path = Path("../docs2")
for p in folder_path.iterdir():
    if p.is_dir():                      # only run on directories
        result = keep_only_key_docs(p, delete_nonimportant=True)
        resultados.append(result)

In [32]:
pd.DataFrame(resultados)['removed'].values[250:300]

array([list(['..\\docs2\\9684721\\1_2.1._CadernoEncargos_e_anexos_VA_2023.pdf', '..\\docs2\\9684721\\3_DEUCP_espd-request.pdf', '..\\docs2\\9684721\\Anuncio13919_2022_DRE210_31out2022_DRE.pdf']),
       list(['..\\docs2\\9689626\\Anexo IV - declaraç╞o de preços unitários.xlsx', '..\\docs2\\9689626\\anuncio.pdf']),
       list(['..\\docs2\\9692428\\2_2022_081CP_PP_Reagentes_an_lise_de_guas.pdf', '..\\docs2\\9692428\\CCP2021A0141953244.pdf']),
       list(['..\\docs2\\9697589\\2_2022_081CP_PP_Reagentes_an_lise_de_guas.pdf', '..\\docs2\\9697589\\CCP2021A0141953244.pdf']),
       list(['..\\docs2\\9698256\\2_2022_081CP_PP_Reagentes_an_lise_de_guas.pdf', '..\\docs2\\9698256\\CCP2021A0141953244.pdf']),
       list(['..\\docs2\\9709215\\1_espd-request.xml', '..\\docs2\\9709215\\Anuncio 14513_2022.pdf', '..\\docs2\\9709215\\Lista.pdf', '..\\docs2\\9709215\\Minuta do anúncio.pdf']),
       list(['..\\docs2\\9709256\\1_espd-request.xml', '..\\docs2\\9709256\\Anuncio 14513_2022.pdf', '..\\docs2\\

### Extrair o pdf do anúncio e do contrato

In [ ]:
# merge o ids extraidos com o código dos scrape e depois correr na função acima

contracts_anun_contr = contracts_scraped.merge(contratos_niclas[['ids_extraidos_docs', 'contract_id']], how='left', left_on = 'contract_id', right_on='contract_id')

In [ ]:
# mergo do que quero extrair com o dataset do niclas porque tem o doc ID
contracts_anun_contr = not_extracted.merge(contratos_niclas[['ids_extraidos_docs', 'contract_id']], how='left', left_on = 'contract_id', right_on='contract_id')

In [ ]:
results_anunc_contrato = [download_contract_pdfs(row, out_folder = "../docs2") for _, row in contracts_anun_contr.iterrows()]

### Código reciclado

In [ ]:
def pecas_link_base(contract_id: int, headless: bool = False, timeout: int = 60, max_tries: int = 5):
    """
    Vai à página do contrato, tenta ler o link em:
      td[data-title="Peças do procedimento"] a
    Se não houver, devolve tem_link=False.
    Recarrega a página com backoff quando a tabela não carrega.
    """
    url = f"https://www.base.gov.pt/Base4/pt/detalhe/?type=contratos&id={contract_id}"
    opts = webdriver.ChromeOptions()
    if headless: opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1366,900")

    drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    try:
        backoff = 2.0
        for attempt in range(1, max_tries + 1):
            drv.get(url)

            # espera o contêiner onde a tabela é injectada via JS
            try:
                WebDriverWait(drv, timeout).until(
                    EC.presence_of_element_located((By.ID, "no-more-tables-mx767"))
                )
            except Exception:
                # se nem o container aparece, faz refresh com backoff
                time.sleep(backoff + random.uniform(0, 1.5))
                backoff *= 1.8
                continue

            # tenta apanhar diretamente o link das peças
            try:
                a = WebDriverWait(drv, timeout).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'td[data-title="Peças do procedimento"] a'))
                )
                href = (a.get_attribute("href") or "").strip()
                return {"tem_link": bool(href), "link": href or None}
            except Exception:
                # se a célula existe mas sem <a>, significa "sem link"
                tds = drv.find_elements(By.CSS_SELECTOR, 'td[data-title="Peças do procedimento"]')
                if tds:
                    txt = (tds[0].text or "").strip()
                    if not txt or txt == "-":
                        return {"tem_link": False, "link": None}

            # não conseguiu agora — espera e tenta outra vez (provável rate-limit / servidor lento)
            time.sleep(backoff + random.uniform(0.5, 2.0))
            backoff *= 1.8

        # esgotou tentativas
        return {"tem_link": False, "link": None}
    finally:
        drv.quit()


# GUARDAR UM FICHEIRO JSONL

def save_result(contract_id: int, result: dict, outfile: Path):
    """
    Acrescenta uma linha ao ficheiro JSONL com:
    {"id": ..., "tem_link": ..., "link": ...}
    """
    data = {"id": contract_id, **result}
    with outfile.open("a", encoding="utf-8") as f:
        f.write(json.dumps(data, ensure_ascii=False) + "\n")
        print('Guardado:', data)


In [ ]:
# --- keep your imports ---
import json, html
import re, time, unicodedata
from pathlib import Path
import urllib.parse as up

# ... keep your existing constants and helpers ...

def _norm(s: str) -> str:
    if not s:
        return ""
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    s = s.lower()
    s = re.sub(r"[\s_]+", " ", s).strip()
    return s

def _want(name: str, type: str) -> bool:
    """
    Heurística simples para 'caderno' ou 'programa'.
    - 'caderno': contém 'caderno' e 'encargos' ou é 'ce' isolado; exclui 'programa' e 'anexo'
    - 'programa': contém 'programa'
    """
    t = _norm(name)
    if type == 'caderno':
        is_ce = re.search(r"\bce\b", t) is not None
        ok = (('caderno' in t and 'encargos' in t) or is_ce)
        bad = ('programa' in t) or ('anexo' in t)
        return ok and not bad
    elif type == 'programa':
        return ('programa' in t)
    else:
        raise ValueError(f"type inválido: {type!r}. Esperado: 'caderno' ou 'programa'.")

def _parse_onclick_concat(onclick: str):
    """
    Collapse string concatenations in onclick and extract query params for DownloadFile.
    Returns (documentFileId, mkey) or (None, None).
    """
    if not onclick:
        return None, None
    s = html.unescape(onclick)  # decode &amp;
    s = re.sub(r"'\s*\+\s*'", "", s)
    s = s.replace("'+ '", "").replace("' +", "")
    m = re.search(r"DownloadFile\?([^'\" ]+)", s)
    if not m:
        return None, None
    qd = dict(up.parse_qsl(m.group(1)))
    return qd.get("documentFileId"), qd.get("mkey")

def _downloads_done(dirpath: Path) -> bool:
    return not any(p.suffix == ".crdownload" for p in dirpath.glob("*"))

def _load_ledger(out_dir: Path) -> set:
    """Keep a tiny JSON ledger of already-downloaded documentFileId values."""
    led = out_dir / "downloaded.json"
    if led.exists():
        try:
            return set(json.loads(led.read_text(encoding="utf-8")))
        except Exception:
            return set()
    return set()

def _save_ledger(out_dir: Path, ids: set):
    (out_dir / "downloaded.json").write_text(json.dumps(sorted(ids)), encoding="utf-8")

def download_vortal_selected_via_browser(row, type: str, out_root: str,
                                         headless: bool = False, timeout: int = 60,
                                         tries: int = 3, max_files: int = 1):
    """
    Downloads up to `max_files` matching documents (default: 1) for this contract row.
    Prevents re-downloads by keeping a ledger of documentFileIds in downloaded.json.
    """
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC

    VORTAL_DOC_NAME = "span.VortalSpan[id*='spnDocumentName']"
    VORTAL_DL_LINK  = "a[id^='lnkDownloadLink']"

    out_dir = Path(out_root) / str(row['id'])
    out_dir.mkdir(parents=True, exist_ok=True)
    out_dir = Path(out_dir).resolve()

    # Load ledger (already downloaded documentFileIds)
    ledger = _load_ledger(out_dir)

    prefs = {
        "download.default_directory": str(out_dir),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "plugins.always_open_pdf_externally": True,
    }
    opts = webdriver.ChromeOptions()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--window-size=1280,900")
    opts.add_argument("--lang=pt-PT")
    opts.add_argument("--disable-popup-blocking")
    opts.add_experimental_option("prefs", prefs)

    drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    try:
        # Allow downloads via CDP (best-effort)
        try:
            drv.execute_cdp_cmd("Page.setDownloadBehavior", {"behavior": "allow", "downloadPath": str(out_dir)})
        except Exception:
            pass

        # load + prep page (with retries)
        for attempt in range(1, tries+1):
            drv.get(row['link'])
            WebDriverWait(drv, timeout).until(lambda d: d.execute_script("return document.readyState") == "complete")

            # cookies
            for how, sel in [
                (By.ID, "onetrust-accept-btn-handler"),
                (By.XPATH, "//button[contains(., 'Aceitar')]"),
                (By.XPATH, "//button[contains(., 'Accept')]"),
                (By.XPATH, "//button[contains(., 'I Agree')]"),
                (By.XPATH, "//div[contains(@class,'cookie')]//button"),
            ]:
                try:
                    btn = WebDriverWait(drv, 3).until(EC.element_to_be_clickable((how, sel)))
                    drv.execute_script("arguments[0].click();", btn)
                    time.sleep(0.2)
                    break
                except Exception:
                    continue

            # open "Documentos" tab
            opened = False
            for xp in (
                "//a[normalize-space()='Documentos']", "//button[normalize-space()='Documentos']",
                "//a[normalize-space()='Documents']",  "//button[normalize-space()='Documents']",
                "//div[contains(@class,'tab')]//a[contains(.,'Documentos')]",
                "//div[contains(@class,'tab')]//a[contains(.,'Documents')]",
            ):
                try:
                    el = drv.find_element(By.XPATH, xp)
                    drv.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
                    time.sleep(0.1)
                    drv.execute_script("arguments[0].click();", el)
                    time.sleep(0.2)
                    opened = True
                    break
                except Exception:
                    pass

            # scroll to load
            last_h = 0
            for _ in range(6):
                drv.execute_script("window.scrollBy(0, document.body.scrollHeight);")
                time.sleep(0.25)
                h = drv.execute_script("return document.body.scrollHeight")
                if h == last_h: break
                last_h = h

            try:
                WebDriverWait(drv, timeout).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, VORTAL_DOC_NAME)))
                if drv.find_elements(By.CSS_SELECTOR, VORTAL_DOC_NAME):
                    break
            except Exception:
                if attempt == tries:
                    (out_dir/"vortal_debug.png").write_bytes(drv.get_screenshot_as_png())
                    (out_dir/"vortal_debug.html").write_text(drv.page_source, encoding="utf-8")
                    return {"ok": False, "reason": "doc_names_not_found",
                            "debug": [str(out_dir/'vortal_debug.png'), str(out_dir/'vortal_debug.html')]}
                time.sleep(1.5 * attempt)

        # Map index suffix -> visible name
        idx_to_name = {}
        for s in drv.find_elements(By.CSS_SELECTOR, VORTAL_DOC_NAME):
            sid = s.get_attribute("id") or ""
            m = re.search(r"_(\d+)$", sid)
            idx = m.group(1) if m else sid
            idx_to_name[idx] = (s.text or "").strip()

        # Collect anchors and pair with their doc ids via onclick
        candidates = []
        for a in drv.find_elements(By.CSS_SELECTOR, VORTAL_DL_LINK):
            aid = a.get_attribute("id") or ""
            m = re.search(r"_(\d+)$", aid)
            idx = m.group(1) if m else aid
            name = idx_to_name.get(idx, "").strip()
            if not name:
                continue
            if not _want(name, type=type):
                continue
            onclick = a.get_attribute("onclick") or ""
            doc_id, mkey = _parse_onclick_concat(onclick)
            candidates.append({"name": name, "a": a, "doc_id": doc_id, "mkey": mkey})

        # Remove ones already downloaded (using the ledger)
        targets = [c for c in candidates if c["doc_id"] and c["doc_id"] not in ledger]

        if not targets:
            return {"ok": True, "matched": 0, "files": [], "note": "no new matching names (all already downloaded or none match)"}

        # First-match only (or up to max_files)
        targets = targets[:max_files]

        # Prepare before/after listing
        before = {p.name for p in out_dir.glob("*") if p.is_file()}

        # Build direct URL when possible, else click
        base = f"{up.urlparse(row['link']).scheme}://{up.urlparse(row['link']).netloc}"
        for c in targets:
            if c["doc_id"] and c["mkey"]:
                dl_url = f"{base}/PRODPublic/Tendering/OpportunityDetail/DownloadFile?documentFileId={c['doc_id']}&mkey={c['mkey']}"
                drv.get(dl_url)
            else:
                drv.execute_script("arguments[0].scrollIntoView({block:'center'});", c["a"])
                drv.execute_script("arguments[0].click();", c["a"])
            time.sleep(0.7)

        # wait for downloads to finish
        t0 = time.time()
        max_wait = max(30, 15 * len(targets))
        while time.time() - t0 < max_wait:
            time.sleep(0.5)
            if _downloads_done(out_dir):
                break

        after = {p.name for p in out_dir.glob("*") if p.is_file()}
        new_files = sorted(list(after - before))

        # Update ledger with the doc_ids we attempted (only if new files appeared)
        for c in targets:
            if c["doc_id"]:
                ledger.add(c["doc_id"])
        _save_ledger(out_dir, ledger)

        return {"ok": True, "matched": len(targets), "files": [str(out_dir / f) for f in new_files]}

    finally:
        drv.quit()


In [ ]:
# import os, re, zipfile
# from pathlib import Path
# import requests
# from urllib.parse import urlparse
# from requests.adapters import HTTPAdapter
# from urllib3.util.retry import Retry

# def _safe_name(s: str) -> str:
#     s = s.strip() or "file"
#     return re.sub(r'[\\/*?:"<>|]+', "_", s)

# def _unique_path(base_dir: Path, name: str) -> Path:
#     p = base_dir / name
#     if not p.exists(): return p
#     stem, ext = Path(name).stem, Path(name).suffix
#     i = 1
#     while True:
#         q = base_dir / f"{stem} ({i}){ext}"
#         if not q.exists(): return q
#         i += 1

# def download_pecas_flat(row, out_root="../docs", timeout=60):
#     out_dir = Path(out_root) / str(row['id'])
#     out_dir.mkdir(parents=True, exist_ok=True)

#     # session with retries
#     sess = requests.Session()
#     sess.headers.update({
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36",
#         "Referer": "https://www.acingov.pt/",
#         "Accept": "*/*",
#     })
#     sess.mount("https://", HTTPAdapter(max_retries=Retry(total=3, backoff_factor=0.5,
#                                                         status_forcelist=[429, 500, 502, 503, 504])))

#     r = sess.get(row['link'], stream=True, timeout=timeout)
#     r.raise_for_status()

#     # best-effort filename from headers or URL tail
#     fname = None
#     cd = r.headers.get("Content-Disposition", "")
#     m = re.search(r'filename\*?=(?:UTF-8\'\')?"?([^";]+)"?', cd)
#     if m:
#         fname = m.group(1)
#     if not fname:
#         tail = Path(urlparse(row['link']).path).name or "pecas"
#         fname = tail
#     if "." not in fname:
#         ctype = (r.headers.get("Content-Type") or "").lower()
#         if "zip" in ctype: ext = ".zip"
#         elif "pdf" in ctype: ext = ".pdf"
#         elif "jnlp" in ctype: ext = ".jnlp"
#         else: ext = ".bin"
#         fname += ext
#     fname = _safe_name(fname)

#     # save to disk
#     tmp_path = _unique_path(out_dir, fname)
#     with open(tmp_path, "wb") as f:
#         for chunk in r.iter_content(1024 * 64):
#             if chunk: f.write(chunk)

#     saved = []
#     # If zip: extract flat into out_dir and delete the original zip
#     if tmp_path.suffix.lower() == ".zip":
#         try:
#             with zipfile.ZipFile(tmp_path, "r") as zf:
#                 for info in zf.infolist():
#                     if info.is_dir():
#                         continue
#                     # flatten: take only basename, drop internal folders
#                     name = _safe_name(Path(info.filename).name)
#                     dest = _unique_path(out_dir, name)
#                     # extract to bytes, then write (avoids path traversal)
#                     with zf.open(info, "r") as src, open(dest, "wb") as dst:
#                         dst.write(src.read())
#                     saved.append(str(dest))
#         finally:
#             # remove the original zip regardless of extraction result
#             try: tmp_path.unlink()
#             except Exception: pass
#     else:
#         saved.append(str(tmp_path))

#     return {"ok": True, "contract_id": row['id'], "files": saved}

In [ ]:
# import os, re, zipfile, time
# from pathlib import Path
# import requests
# from urllib.parse import urlparse, unquote
# from requests.adapters import HTTPAdapter
# from urllib3.util.retry import Retry

# def _safe_name(s: str) -> str:
#     s = s.strip() or "file"
#     return re.sub(r'[\\/*?:"<>|]+', "_", s)

# def _unique_path(base_dir: Path, name: str) -> Path:
#     p = base_dir / name
#     if not p.exists(): return p
#     stem, ext = Path(name).stem, Path(name).suffix
#     i = 1
#     while True:
#         q = base_dir / f"{stem} ({i}){ext}"
#         if not q.exists(): return q
#         i += 1

# def _make_session():
#     sess = requests.Session()
#     sess.headers.update({
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36",
#         "Referer": "https://www.acingov.pt/",
#         "Accept": "*/*",
#     })
#     retries = Retry(
#         total=5, connect=5, read=5,
#         backoff_factor=0.6,
#         status_forcelist=[429, 500, 502, 503, 504],
#         allowed_methods={"GET", "HEAD", "OPTIONS"},
#         respect_retry_after_header=True,
#         raise_on_redirect=True
#     )
#     adapter = HTTPAdapter(max_retries=retries, pool_connections=20, pool_maxsize=20)
#     sess.mount("https://", adapter)
#     sess.mount("http://", adapter)
#     return sess

# def _pick_filename(resp, url_tail: str) -> str:
#     # Try Content-Disposition (handles RFC5987 filename*=UTF-8'')
#     cd = resp.headers.get("Content-Disposition", "")
#     m = re.search(r'filename\*?=(?:UTF-8\'\')?"?([^";]+)"?', cd)
#     if m:
#         fname = unquote(m.group(1))
#     else:
#         fname = Path(url_tail).name or "pecas"
#     if "." not in fname:
#         ctype = (resp.headers.get("Content-Type") or "").lower()
#         if "zip" in ctype: ext = ".zip"
#         elif "pdf" in ctype: ext = ".pdf"
#         elif "jnlp" in ctype: ext = ".jnlp"
#         else: ext = ".bin"
#         fname += ext
#     return _safe_name(fname)

# def _download_with_resume(sess, url, out_path: Path,
#                           chunk_size=64 * 1024,
#                           connect_timeout=5,
#                           read_timeout=180,
#                           max_attempts=8,
#                           max_total_time=None):
#     """
#     Stream download that survives read timeouts by resuming with HTTP Range.
#     """
#     out_path.parent.mkdir(parents=True, exist_ok=True)
#     tmp_path = out_path.with_suffix(out_path.suffix + ".part")

#     pos = tmp_path.stat().st_size if tmp_path.exists() else 0
#     start_time = time.time()
#     attempts = 0
#     etag = None
#     headers = {}

#     # First request also used to pick filename when caller hasn't decided yet
#     while True:
#         if max_total_time and (time.time() - start_time) > max_total_time:
#             raise TimeoutError(f"Aborted after {max_total_time}s total time: {url}")

#         range_hdr = {"Range": f"bytes={pos}-"} if pos > 0 else {}
#         resp = sess.get(url, stream=True, headers=range_hdr | headers,
#                         timeout=(connect_timeout, read_timeout))

#         # If we tried to resume but server ignored Range, restart from scratch
#         if pos > 0 and resp.status_code == 200:
#             pos = 0
#             tmp_path.write_bytes(b"")

#         # Capture ETag so we can protect resumes
#         if etag is None and resp.headers.get("ETag"):
#             etag = resp.headers["ETag"]
#             headers["If-Range"] = etag

#         try:
#             with tmp_path.open("ab") as f:
#                 for chunk in resp.iter_content(chunk_size):
#                     if chunk:
#                         f.write(chunk)
#                         pos += len(chunk)
#             break  # success
#         except (requests.ReadTimeout, requests.ConnectionError, requests.ChunkedEncodingError) as e:
#             attempts += 1
#             resp.close()
#             if attempts > max_attempts:
#                 raise ConnectionError(f"Exceeded retries while downloading {url}") from e
#             # small exponential backoff
#             time.sleep(min(0.25 * (2 ** min(attempts, 6)), 10))
#             continue
#         finally:
#             resp.close()

#     tmp_path.replace(out_path)
#     return out_path

# def download_pecas_flat(row, out_root="../docs", timeout=180):
#     """
#     timeout can be:
#       - int/float -> used as read timeout, connect timeout defaults to 5s
#       - (connect_timeout, read_timeout) tuple
#     """
#     out_dir = Path(out_root) / str(row['id'])
#     out_dir.mkdir(parents=True, exist_ok=True)

#     # session with robust retries
#     sess = _make_session()

#     url = row['link']
#     connect_t, read_t = (5, timeout) if not isinstance(timeout, (tuple, list)) else tuple(timeout)

#     # HEAD (optional) – helps with early filename detection & quick failure
#     try:
#         h = sess.head(url, allow_redirects=True, timeout=(connect_t, min(read_t, 10)))
#         h.raise_for_status()
#         fname = _pick_filename(h, urlparse(url).path)
#     except Exception:
#         # fallback to GET-derived name later
#         fname = None

#     # If we didn't get a name from HEAD, use a short GET to decide name
#     if not fname:
#         g = sess.get(url, stream=True, timeout=(connect_t, min(read_t, 15)))
#         g.raise_for_status()
#         fname = _pick_filename(g, urlparse(url).path)
#         g.close()

#     fname = _safe_name(fname)
#     out_path = _unique_path(out_dir, fname)

#     # Do the resilient download (with resume on stall)
#     _download_with_resume(
#         sess, url, out_path,
#         chunk_size=64 * 1024,
#         connect_timeout=connect_t,
#         read_timeout=read_t,
#         max_attempts=8,
#         max_total_time=None,   # or set a ceiling (e.g., 30*60)
#     )

#     saved = []

#     # If zip: extract flat into out_dir and delete the original zip
#     if out_path.suffix.lower() == ".zip":
#         try:
#             with zipfile.ZipFile(out_path, "r") as zf:
#                 for info in zf.infolist():
#                     if info.is_dir():
#                         continue
#                     name = _safe_name(Path(info.filename).name)
#                     dest = _unique_path(out_dir, name)
#                     # read -> write to prevent path traversal
#                     with zf.open(info, "r") as src, open(dest, "wb") as dst:
#                         dst.write(src.read())
#                     saved.append(str(dest))
#         finally:
#             try: out_path.unlink()
#             except Exception: pass
#     else:
#         saved.append(str(out_path))

#     return {"ok": True, "contract_id": row['id'], "files": saved}


In [ ]:
# import io, zipfile
# from pathlib import Path


# def _extract_zip_flat(zip_path: Path, out_dir: Path,
#                       max_depth: int = 2,
#                       depth: int = 0,
#                       max_files: int = 5000,
#                       max_uncompressed_bytes: int = 2_000_000_000):
#     """
#     Extracts `zip_path` into `out_dir`, flattening paths (keeps only base file names),
#     and recursively extracts nested zips up to `max_depth`.

#     Safety:
#       - Avoids Zip Slip by ignoring inner directories and using only basename.
#       - Caps total files and total uncompressed size (best-effort).
#       - Limits recursion depth to avoid endless nesting.

#     Returns: list of saved file paths (strings).
#     """
#     saved = []
#     file_count = 0
#     total_uncompressed = 0

#     def _extract_stream(zf: zipfile.ZipFile, out_dir: Path, depth: int):
#         nonlocal file_count, total_uncompressed, saved

#         for info in zf.infolist():
#             if info.is_dir():
#                 continue

#             # zip-bomb guards
#             total_uncompressed += info.file_size
#             file_count += 1
#             if file_count > max_files:
#                 raise RuntimeError(f"Aborting: too many files ({file_count}>{max_files}) in nested zips.")
#             if total_uncompressed > max_uncompressed_bytes:
#                 raise RuntimeError(f"Aborting: uncompressed size limit exceeded ({total_uncompressed} bytes).")

#             name = _safe_name(Path(info.filename).name)
#             dest = _unique_path(out_dir, name)

#             with zf.open(info, "r") as src:
#                 data = src.read()

#             # nested zip?
#             if name.lower().endswith(".zip") and depth < max_depth:
#                 try:
#                     with zipfile.ZipFile(io.BytesIO(data), "r") as nested:
#                         _extract_stream(nested, out_dir, depth + 1)
#                     # don't store the intermediate .zip if we could unpack it
#                     continue
#                 except zipfile.BadZipFile:
#                     # not actually a valid zip; fall through and write bytes
#                     pass

#             with open(dest, "wb") as f:
#                 f.write(data)
#             saved.append(str(dest))

#     with zipfile.ZipFile(zip_path, "r") as zf:
#         _extract_stream(zf, out_dir, depth)

#     # remove the original .zip after successful extraction
#     try:
#         zip_path.unlink()
#     except Exception:
#         pass

#     return saved


In [ ]:
# import os, re, zipfile, time
# from pathlib import Path
# import requests
# from urllib.parse import urlparse, unquote
# from requests.adapters import HTTPAdapter
# from urllib3.util.retry import Retry

# def _safe_name(s: str) -> str:
#     s = s.strip() or "file"
#     return re.sub(r'[\\/*?:"<>|]+', "_", s)

# def _unique_path(base_dir: Path, name: str) -> Path:
#     p = base_dir / name
#     if not p.exists(): return p
#     stem, ext = Path(name).stem, Path(name).suffix
#     i = 1
#     while True:
#         q = base_dir / f"{stem} ({i}){ext}"
#         if not q.exists(): return q
#         i += 1

# def _make_session():
#     sess = requests.Session()
#     sess.headers.update({
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36",
#         "Referer": "https://www.acingov.pt/",
#         "Accept": "*/*",
#     })
#     retries = Retry(
#         total=5, connect=5, read=5,
#         backoff_factor=0.6,
#         status_forcelist=[429, 500, 502, 503, 504],
#         allowed_methods={"GET", "HEAD", "OPTIONS"},
#         respect_retry_after_header=True,
#         raise_on_redirect=True
#     )
#     adapter = HTTPAdapter(max_retries=retries, pool_connections=20, pool_maxsize=20)
#     sess.mount("https://", adapter)
#     sess.mount("http://", adapter)
#     return sess

# def _pick_filename(resp, url_tail: str) -> str:
#     # Try Content-Disposition (handles RFC5987 filename*=UTF-8'')
#     cd = resp.headers.get("Content-Disposition", "")
#     m = re.search(r'filename\*?=(?:UTF-8\'\')?"?([^";]+)"?', cd)
#     if m:
#         fname = unquote(m.group(1))
#     else:
#         fname = Path(url_tail).name or "pecas"
#     if "." not in fname:
#         ctype = (resp.headers.get("Content-Type") or "").lower()
#         if "zip" in ctype: ext = ".zip"
#         elif "pdf" in ctype: ext = ".pdf"
#         elif "jnlp" in ctype: ext = ".jnlp"
#         else: ext = ".bin"
#         fname += ext
#     return _safe_name(fname)

# def _download_with_resume(sess, url, out_path: Path,
#                           chunk_size=64 * 1024,
#                           connect_timeout=5,
#                           read_timeout=180,
#                           max_attempts=8,
#                           max_total_time=None):
#     """
#     Stream download that survives read timeouts by resuming with HTTP Range.
#     """
#     out_path.parent.mkdir(parents=True, exist_ok=True)
#     tmp_path = out_path.with_suffix(out_path.suffix + ".part")

#     pos = tmp_path.stat().st_size if tmp_path.exists() else 0
#     start_time = time.time()
#     attempts = 0
#     etag = None
#     headers = {}

#     # First request also used to pick filename when caller hasn't decided yet
#     while True:
#         if max_total_time and (time.time() - start_time) > max_total_time:
#             raise TimeoutError(f"Aborted after {max_total_time}s total time: {url}")

#         range_hdr = {"Range": f"bytes={pos}-"} if pos > 0 else {}
#         resp = sess.get(url, stream=True, headers=range_hdr | headers,
#                         timeout=(connect_timeout, read_timeout))

#         # If we tried to resume but server ignored Range, restart from scratch
#         if pos > 0 and resp.status_code == 200:
#             pos = 0
#             tmp_path.write_bytes(b"")

#         # Capture ETag so we can protect resumes
#         if etag is None and resp.headers.get("ETag"):
#             etag = resp.headers["ETag"]
#             headers["If-Range"] = etag

#         try:
#             with tmp_path.open("ab") as f:
#                 for chunk in resp.iter_content(chunk_size):
#                     if chunk:
#                         f.write(chunk)
#                         pos += len(chunk)
#             break  # success
#         except (requests.ReadTimeout, requests.ConnectionError, requests.ChunkedEncodingError) as e:
#             attempts += 1
#             resp.close()
#             if attempts > max_attempts:
#                 raise ConnectionError(f"Exceeded retries while downloading {url}") from e
#             # small exponential backoff
#             time.sleep(min(0.25 * (2 ** min(attempts, 6)), 10))
#             continue
#         finally:
#             resp.close()

#     tmp_path.replace(out_path)
#     return out_path

# def download_pecas_flat(row, out_root="../docs", timeout=180):
#     """
#     timeout can be:
#       - int/float -> used as read timeout, connect timeout defaults to 5s
#       - (connect_timeout, read_timeout) tuple
#     """
#     out_dir = Path(out_root) / str(row['id'])
#     out_dir.mkdir(parents=True, exist_ok=True)

#     # session with robust retries
#     sess = _make_session()

#     url = row['link']
#     connect_t, read_t = (5, timeout) if not isinstance(timeout, (tuple, list)) else tuple(timeout)

#     # HEAD (optional) – helps with early filename detection & quick failure
#     try:
#         h = sess.head(url, allow_redirects=True, timeout=(connect_t, min(read_t, 10)))
#         h.raise_for_status()
#         fname = _pick_filename(h, urlparse(url).path)
#     except Exception:
#         # fallback to GET-derived name later
#         fname = None

#     # If we didn't get a name from HEAD, use a short GET to decide name
#     if not fname:
#         g = sess.get(url, stream=True, timeout=(connect_t, min(read_t, 15)))
#         g.raise_for_status()
#         fname = _pick_filename(g, urlparse(url).path)
#         g.close()

#     fname = _safe_name(fname)
#     out_path = _unique_path(out_dir, fname)

#     # Do the resilient download (with resume on stall)
#     _download_with_resume(
#         sess, url, out_path,
#         chunk_size=64 * 1024,
#         connect_timeout=connect_t,
#         read_timeout=read_t,
#         max_attempts=8,
#         max_total_time=None,   # or set a ceiling (e.g., 30*60)
#     )

#     saved = []

#     # If zip: extract flat into out_dir and delete the original zip
#     if out_path.suffix.lower() == ".zip":
#         try:
#             saved = _extract_zip_flat(out_path, out_dir, max_depth=3)
#         except Exception as e:
#             # If it fails, keep the original zip so you can inspect it
#             saved = [str(out_path)]
#     else:
#         saved = [str(out_path)]

#     return {"ok": True, "contract_id": row['id'], "files": saved}


In [ ]:
contracts_download[:2].apply(download_pecas_flat, axis=1, out_root="../docs")

5      {'ok': True, 'contract_id': 9722182, 'files': ...
16     {'ok': True, 'contract_id': 9946477, 'files': ...
26     {'ok': True, 'contract_id': 10020105, 'files':...
27     {'ok': True, 'contract_id': 10020164, 'files':...
28     {'ok': True, 'contract_id': 10034714, 'files':...
                             ...                        
783    {'ok': True, 'contract_id': 9980931, 'files': ...
787    {'ok': True, 'contract_id': 9980868, 'files': ...
790    {'ok': True, 'contract_id': 9694357, 'files': ...
793    {'ok': True, 'contract_id': 9694859, 'files': ...
796    {'ok': True, 'contract_id': 9980968, 'files': ...
Length: 320, dtype: object